In [1]:
import torch
# import numpy as np
# import matplotlib.pyplot as plt

# from datasets import get_dataset
from basics import train, get_data, accuracy
from extensions import ExtendedData
from torch_geometric.datasets import StochasticBlockModelDataset
from synthetic_data import SBM
# import torch_geometric.utils 
from np_gnn.models import NPGNN
# from optimize import evaluate_model, evaluate_params

# import platform
# platform.system()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n = 1200
# Create a symmetric tensor with p on the diagonal and q off diagonal

p = 0.1
q = 0.02
K = 3

# Create a tensor filled with q
B = torch.full((K, K), q)
# Fill the diagonal with p
for i in range(K):
    B[i, i] = p

B = B * 0.1

data = StochasticBlockModelDataset('data/SBM', [n//3, n//3, n//3], B, num_channels=3, class_sep=.1)[0]
# data = SBM('data/SBM', [n//3, n//3, n//3], B, num_channels=3, class_sep=2)[0]

# data = get_data('squirrel')

data = ExtendedData.from_dict(data.to_dict())
data.create_masks('balanced')
# data.to('cpu')

# Print mean degree
print(torch.mean(data.get_degrees()))
data.plot_features_3d(use_svd=True)


tensor(41.7368, dtype=torch.float64)


In [2]:
# !pip install git+https://github.com/aaamini/graph_visualizer.git
from graph_visualizer import visualize_graph

# Visualize the graph
visualize_graph(data)


Gtk-Message: 16:14:41.380: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:14:41.380: Failed to load module "canberra-gtk-module"
[2:2:0518/161441.532823:ERROR:nacl_fork_delegate_linux.cc(313)] Bad NaCl helper startup ack (0 bytes)


Stub sandbox ignoring command: /app/extra/nacl_helper
Opening in existing browser session.


Gtk-Message: 16:14:41.596: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:14:41.597: Failed to load module "canberra-gtk-module"
libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)


In [3]:
from tqdm import tqdm

torch.manual_seed(42)

data = data.to(device)
adj = data.get_adjacency_matrix().to_dense()
n_feats = data.x.shape[1]
n_out = len(data.y.unique())

lr = 1e-2
wd = 5e-4
nreps = data.masks['train'].shape[1]
n_iter = 1500

val_results = []
tst_results = []

for rep in tqdm(range(nreps), desc='Training Repetitions'):
    model = NPGNN(adj, n_feats, n_out)
    best_acc, test_acc, iterations = train(
        model, data, rep, n_iter, lr, wd
    )

    val_results.append(best_acc)
    tst_results.append(test_acc)
    tqdm.write(f"Rep {rep+1}/{nreps} - val = {best_acc:.3f}, tst = {test_acc:.3f}, # iter = {iterations+1:4d}")

mean_val = np.mean(val_results)
std_val = np.std(val_results)
mean_tst = np.mean(tst_results)
std_tst = np.std(tst_results)
tqdm.write(f"\n---\nmean val = {mean_val:.3f} ± {std_val:.3f}, mean tst = {mean_tst:.3f} ± {std_tst:.3f}")


Training Repetitions:   0%|          | 0/10 [00:00<?, ?it/s]/home/arash/miniforge3/envs/torch/lib/python3.11/site-packages/torch/optim/optimizer.py:243: UserWarning:

'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'

Training Repetitions:  10%|█         | 1/10 [00:56<08:25, 56.20s/it]

Rep 1/10 - val = 0.365, tst = 0.371, # iter =  511


Training Repetitions:  20%|██        | 2/10 [01:52<07:29, 56.25s/it]

Rep 2/10 - val = 0.346, tst = 0.371, # iter =  563
